In [1]:
%cd hypernet

/home/z1157095/hypernet-cnn/hypernet


In [2]:
from dotenv import load_dotenv
load_dotenv()

import random
import os

In [3]:
from comet_ml import Experiment, Optimizer

import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import torch.utils.data as data_utils
import pandas as pd
from collections import defaultdict

torch.set_default_dtype(torch.float32)

In [4]:
from torchsummary import summary
import matplotlib.pyplot as plt
from tqdm import tqdm, trange

In [5]:
import tabular_hypernet as hp

In [6]:
hp.training_utils.get_dataset

<function tabular_hypernet.training_utils.get_dataset(size=60000, masked=False, mask_no=200, mask_size=700, shared_mask=False, batch_size=32, test_batch_size=32)>

In [7]:
os.environ['COMET_KEY'] = 'UXrV5UxyhTK3cyQNG6BDuc4bE'
os.environ.get("COMET_KEY")

'UXrV5UxyhTK3cyQNG6BDuc4bE'

In [8]:
# TODO

# TabSSLCrossEntropyLoss if confidence is high for both masks, add selfsl loss

# Verify if masks always divided into pairs the same way

In [9]:
class TabSSLCrossEntropyLoss(torch.nn.Module):
    def __init__(self, beta=0.1, unsup_target_wrapper=torch.nn.functional.softmax, threshold=None):
        super(TabSSLCrossEntropyLoss, self).__init__()
        
        self.y_f1 = torch.nn.CrossEntropyLoss()
        self.y_f2 = torch.nn.CrossEntropyLoss()
        
        self.f1_f2 = torch.nn.CrossEntropyLoss()
        self.f2_f1 = torch.nn.CrossEntropyLoss()
        
        self.beta = beta
        self.unsup_target_wrapper = unsup_target_wrapper
        self.threshold = threshold
        
    def is_observ_above_threshold(self, data):
        mask = torch.any(data >= self.threshold, dim=1)
        
        return mask
        
    
    def forward(self, sup_input, unsup_input):
        sup_outputs1, sup_outputs2, sup_labels = sup_input
        unsup_outputs1, unsup_outputs2 = unsup_input
        
        self.supervised_loss = self.y_f1(sup_outputs1, sup_labels) + self.y_f2(sup_outputs2, sup_labels)
        
        self.self_supervised_loss = 0
        if self.beta:
            print('Beta')
            if self.threshold:
                unsup_outputs1_target = torch.nn.functional.softmax(unsup_outputs1, dim=1)
                mask1 = self.is_observ_above_threshold(unsup_outputs1_target)

                if len(unsup_outputs1_target[mask1]):
                    unsup_outputs1_target = torch.argmax(unsup_outputs1_target[mask1], dim=1)
                    self.self_supervised_loss += self.f2_f1(unsup_outputs2[mask1], unsup_outputs1_target)

                unsup_outputs2_target = torch.nn.functional.softmax(unsup_outputs2, dim=1)
                mask2 = self.is_observ_above_threshold(unsup_outputs2_target)

                if len(unsup_outputs2_target[mask2]):
                    unsup_outputs2_target = torch.argmax(unsup_outputs2_target[mask2], dim=1)
                    self.self_supervised_loss += self.f1_f2(unsup_outputs1[mask2], unsup_outputs2_target)

            else:
                self.self_supervised_loss = self.f1_f2(unsup_outputs1, self.unsup_target_wrapper(unsup_outputs2, dim=1)) \
                                    + self.f2_f1(unsup_outputs2, self.unsup_target_wrapper(unsup_outputs1, dim=1))      
        
        return self.supervised_loss + self.beta * self.self_supervised_loss


In [10]:
class VarianceWithThresholdMasksSelector():
    def __init__(self, data, mask_size, threshold=0.2):
        
        self.mask_size = mask_size
        
        trainset, _ = data
        stacked_set = torch.stack([x for x, _ in trainset])
        
        feature_variances = torch.var(stacked_set, dim=0)
        self.feat_above_thresh = (feature_variances > threshold).nonzero()[:, 0]
        
    
    def _random_choice_mask(self, mask_size):
        tmp = np.random.choice(self.feat_above_thresh, mask_size, replace=False)
        tmp.sort()
        return tmp
        
    def __call__(self, model, count):
        print(model)
        print(self)
        
        tmp = np.array([model.template.copy() for _ in range(count)])
        
        for i in range(count):
            mask = self._random_choice_mask(model.mask_size)
            tmp[i, mask] = 1
            
        masks = torch.from_numpy(tmp).to(torch.float32).to(model.device)
        return masks
        
        

In [11]:
seed = 5

In [12]:
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

In [13]:
mask_size = 20


In [14]:
dataset = hp.semisl.get_train_test_sets()

In [15]:
# mask_selector = VarianceWithThresholdMasksSelector(dataset, mask_size=mask_size, threshold=selector_threshold)

In [16]:
# mask_selector.feat_above_thresh

In [17]:
epochs = 100

masks_no = 50


results = defaultdict(list)
size = (100, 59900)


##### Variance-based feature selection

In [18]:
for beta in [0.]:
    for threshold_var in [0., 0.1, 0.2, 0.4, 0.7, 1]:
        for lr in [3e-5]:
            mask_selector = VarianceWithThresholdMasksSelector(dataset, mask_size=mask_size, threshold=threshold_var)
            
            criterion = TabSSLCrossEntropyLoss(beta=beta, threshold=None)
            
            np.random.seed(seed)
            torch.manual_seed(seed)
            random.seed(seed)
            
            # dataset
            
            
            hypernet = hp.Hypernetwork(
                architecture=torch.nn.Sequential(
                    torch.nn.Linear(784, 64), 
                    torch.nn.ReLU(),
                    torch.nn.Linear(64, 256),
                    torch.nn.ReLU(),
                    torch.nn.Linear(256, 128),
                ),
                target_architecture=[(mask_size, 10), (10, 10)],
                test_nodes=masks_no,
            ).cuda()
            hypernet._create_mask = None
            hypernet.test_mask = mask_selector(hypernet, hypernet.test_nodes)

            hypernet = hypernet.train()
            
            
            
            optimizer = torch.optim.Adam(hypernet.parameters(), lr=lr)

            # loaders
            sup_trainloader, unsup_trainloader, testloader = hp.semisl.get_dataloaders(dataset=dataset, size=size, batch_size=32, test_batch_size=64)
            trainloader = hp.semisl.TrainDataLoaderSemi(sup_trainloader, unsup_trainloader)

            results[size].append(hp.semisl.train_semisl(hypernet,
                                              optimizer,
                                              criterion,
                                              (trainloader, testloader), 
                                              size,
                                              epochs,
                                              masks_no,
                                              changing_beta=None,
                                              log_to_comet=True,
                                              tags=['mask variance selection with threshold'],
                                              description="""
                                              masks are selected based on feature variance (above threshold) 
                                              """,
                                            log_params={'seed': seed, 'selector_threshold': threshold_var}
                                            ))
            

torch.Size([1, 128])


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


Hypernetwork(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (output_layer): Linear(in_features=128, out_features=320, bias=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)


COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/66bd652fd27f4433ac674d1924b5cce0

100%|███████████████| 100/100 [31:36<00:00, 18.97s/it, loss=1.79, test_acc=67.6]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading metrics, params, and assets to Comet before program termination (may take several seconds)
COMET INFO: The Python SDK has 3600 seconds to finish before aborting...
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


torch.Size([1, 128])
Hypernetwork(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (output_layer): Linear(in_features=128, out_features=320, bias=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)


...

COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/876b990a42d3402d9298437f68342a6d

100%|████████████████| 100/100 [31:44<00:00, 19.04s/it, loss=1.8, test_acc=66.9]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


torch.Size([1, 128])
Hypernetwork(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (output_layer): Linear(in_features=128, out_features=320, bias=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)


COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/0fd8b8e0ac0345b19ba6201b0d9d0374

100%|███████████████| 100/100 [31:59<00:00, 19.19s/it, loss=1.79, test_acc=67.8]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


torch.Size([1, 128])
Hypernetwork(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (output_layer): Linear(in_features=128, out_features=320, bias=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)


COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/caad40cba79f4bcaa764236ba66f7e0a

100%|████████████████| 100/100 [31:36<00:00, 18.96s/it, loss=1.8, test_acc=66.8]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages


In [19]:
hypernet_with_selector = hypernet

In [ ]:
m1 = hypernet.test_mask[0]
m1

In [ ]:
torch.sum(m1)

#### No variance-based feature selection

In [42]:
for beta in [0.]:
    for threshold in [0.]:
        for lr in [3e-5]:
            criterion = TabSSLCrossEntropyLoss(beta=beta, threshold=None)
            
            np.random.seed(seed)
            torch.manual_seed(seed)
            random.seed(seed)
            
            # dataset
            
            
            hypernet = hp.Hypernetwork(
                architecture=torch.nn.Sequential(
                    torch.nn.Linear(784, 64), 
                    torch.nn.ReLU(),
                    torch.nn.Linear(64, 256),
                    torch.nn.ReLU(),
                    torch.nn.Linear(256, 128),
                ),
                target_architecture=[(mask_size, 10), (10, 10)],
                test_nodes=masks_no,
            ).cuda()
            
            
            hypernet = hypernet.train()
            
            
            
            optimizer = torch.optim.Adam(hypernet.parameters(), lr=lr)

            # loaders
            sup_trainloader, unsup_trainloader, testloader = hp.semisl.get_dataloaders(dataset=dataset, size=size, batch_size=32, test_batch_size=64)
            trainloader = hp.semisl.TrainDataLoaderSemi(sup_trainloader, unsup_trainloader)

            results[size].append(hp.semisl.train_semisl(hypernet,
                                              optimizer,
                                              criterion,
                                              (trainloader, testloader), 
                                              size,
                                              epochs,
                                              masks_no,
                                              changing_beta=None,
                                              log_to_comet=True,
                                              tags=['mask variance selection with threshold'],
                                              description="""
                                              masks are selected based on feature variance (above threshold) 
                                              """,
                                            log_params={'seed': seed,'selector_threshold': 0}
                                            ))
            

torch.Size([1, 128])


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/5978fdf6426341edb0f1d290a335117c

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [33:23<00:00, 20.04s/it, loss=1.86, test_acc=61.3]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages


In [ ]:
m2 = hypernet.test_mask[0]
m2

In [ ]:
m1==m2

In [ ]:
torch.sum(m2)

In [44]:
hypernet_with_selector.test_mask[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 